# Conhecendo a base de dados

A base de dados a ser explorada será a de imóveis para aluguel no RJ. O CSV está disponível no Teams na aba "Arquivos" do canal Geral, pasta "Material de Aula".

Baixe o arquivo "aluguel.csv" do teams e faça upload aqui no Colab através da aba ao lado "Arquivos", ícone "Fazer upload".
Lembre-se que os arquivos enviados e gerados pelo Colab são apagados quando a sessão em uso expira!

## Importando os dados


In [ ]:
import pandas as pd

In [ ]:
arq = '/content/aluguel.csv'
dados = pd.read_csv(arq, sep=';')
dados

In [ ]:
dados.head(10)

In [ ]:
dados.tail()

In [ ]:
type(dados)

## Características gerais da base de dados

Nesta seção, estudaremos o formato e composição da tabela de dados (DataFrame).
Mostraremos as dimensões da tabela (DataFrame) assim como a forma de selecionar os atributos do DF e verificar seus valores

In [ ]:
dados.shape

In [ ]:
dados.columns

In [ ]:
dados.info()

In [ ]:
dados['Tipo']

In [ ]:
dados['Tipo'].unique()

In [ ]:
dados[['Quartos', 'Valor']]

# Análise exploratória de dados

Nesta seção, responderemos algumas perguntas para que possamos adquirir conhecimentos sobre o dados que nossa base contém.

## Qual o valor médio de aluguel por tipo de imóvel?

In [ ]:
dados.head()

In [ ]:
dados.describe()

In [ ]:
dados.groupby('Tipo').mean(numeric_only=True)

In [ ]:
dados.groupby('Tipo')[['Valor']].mean().sort_values('Valor')

In [ ]:
df_preco_tipo = dados.groupby('Tipo')[['Valor']].mean().sort_values('Valor')

df_preco_tipo.plot(kind='barh', figsize=(7, 5), color ='purple');

## Removendo os imóveis comerciais

Nossa imobiliária trabalha apenas com imóveis residenciais, então, removeremos os tipos não residenciais.

In [ ]:
dados.Tipo.unique()

In [ ]:
imoveis_residenciais = ['Quitinete', 'Casa', 'Apartamento','Casa de Condomínio','Flat', 'Casa de Vila','Terreno Padrão', 'Loft', 'Chácara', 'Loteamento/Condomínio',
       'Sítio', 'Studio']


In [ ]:
df_residenciais = dados.query('@imoveis_residenciais in Tipo')

In [ ]:
df_residenciais.head()

In [ ]:
df_residenciais.Tipo.unique()

In [ ]:
df_preco_tipo = df_residenciais.groupby('Tipo')[['Valor']].mean().sort_values('Valor')

df_preco_tipo.plot(kind='barh', figsize=(7, 5), color ='purple');

## Qual o percentual de cada tipo de imóvel na nossa base de dados?

In [ ]:
df_residenciais.Tipo.unique()

In [ ]:
df_residenciais.Tipo.value_counts(normalize=True)

In [ ]:
df_proporcao = df_residenciais.Tipo.value_counts(normalize=True).to_frame().sort_values('Tipo')
df_proporcao['proportion'] = df_proporcao['proportion']*100
df_proporcao

In [ ]:
df_percentual_tipo = df_residenciais.Tipo.value_counts(normalize=True).to_frame().sort_values('Tipo')

df_percentual_tipo.plot(kind='bar', figsize=(7, 5), color ='green',
                        xlabel = 'Tipos', ylabel = 'Percentual');

### **Selecionando apenas os imóveis do tipo apartamento**

Como 84% dos imóveis são apartamentos, faz sentindo restringir nosso escopo para apenas apartamentos

In [ ]:
df_residenciais.query('Tipo == "Apartamento"')

In [ ]:
df_apto = df_residenciais.query('Tipo == "Apartamento"')
df_apto.head()

# Pré-processamento

## Lidando com amostras duplicadas

In [ ]:
df_apto.duplicated()

In [ ]:
df_apto[df_apto.duplicated()]

In [ ]:
df_apto = df_apto.drop_duplicates()
df_apto.head()

In [ ]:
df_apto.shape

## Lidando com dados nulos

In [ ]:
df_apto.isnull()

In [ ]:
df_apto.isnull().sum()

Deletar apenas os apartamentos sem valor

In [ ]:
df_apto = df_apto.dropna(subset=['Valor'])
df_apto.isnull().sum()

In [ ]:
df_apto.fillna(0)

Os demais apartamentos, vou considerar que os valores de IPTU e Condomninio já estão inclusos no Valor, por isso, quero preencher estes valores com zero (0)

In [ ]:
df_apto = df_apto.fillna(0)

In [ ]:
df_apto.isnull().sum()

## Removendo registros

Por algum motivo, a imobiliária não trabalha com Apartamentos no Centro sem vagas garagem. Precisamos, portanto, remover estas amostras.

In [ ]:
df_apto.query('Vagas == 0 & Bairro == "Centro"')

In [ ]:
df_apto.query('Vagas == 0  & Bairro == "Centro"').index

In [ ]:
indices_a_remover = df_apto.query('Vagas == 0 & Bairro == "Centro"').index

In [ ]:
df_apto.drop(indices_a_remover, axis=0, inplace=True)

In [ ]:
df_apto.query('Vagas == 0  & Bairro == "Centro"')

In [ ]:
df_apto.head()

In [ ]:
df_apto.Tipo.unique()

In [ ]:
df_apto.drop('Tipo', axis=1, inplace=True)

In [ ]:
df_apto.head()

## Filtros

### **1. Apartamentos que possuem `1 quarto` e `aluguel menor que 1200`**



In [ ]:
df_apto['Quartos'] == 1

In [ ]:
selecao1 = df_apto['Quartos'] == 1
df_apto[selecao1]

In [ ]:
selecao2 = df_apto['Valor'] < 1200
df_apto[selecao2]

In [ ]:
selecao_final = (selecao1) & (selecao2)
df_apto[selecao_final]

In [ ]:
df_1 = df_apto[selecao_final]

### **2. `Apartamentos` que possuem pelo menos `2 quartos`, `aluguel menor que 3000` e `area maior que 70`**

In [ ]:
selecao = (df_apto['Quartos'] >= 2) & (df_apto['Valor'] < 3000) & (df_apto['Area'] > 70)
df_apto[selecao]

In [ ]:
df_2 = df_apto[selecao]

## Salvando os dados das nossas duas consultas

In [ ]:
df_1.to_csv('apartamentos-1.csv')

In [ ]:
pd.read_csv('apartamentos-1.csv')

In [ ]:
df_2.to_csv('apartamentos-2.csv', index=False)

In [ ]:
pd.read_csv('apartamentos-2.csv')

In [ ]:
df_apto.to_csv('apartamentos.csv', index=False, sep=';')

In [ ]:
pd.read_csv('apartamentos.csv')

In [ ]:
pd.read_csv('apartamentos.csv', sep=';')

# Manipulando os dados

## Criando colunas numéricas

In [ ]:
df_apto.head()

In [ ]:
df_apto['valor_por_ano'] = (df_apto['Valor'] + df_apto['Condominio']) * 12 + df_apto['IPTU']
df_apto.head()

In [ ]:
df_apto['valor_medio_por_mes'] = df_apto['valor_por_ano'] / 12
df_apto.head()

## Criando colunas categóricas

In [ ]:
df_apto['Possui_suite'] = df_apto['Suites'].apply(lambda x: "Sim" if x > 0 else "Não")
df_apto.head()

In [ ]:
dados.to_csv('apartamentos.csv', index=False, sep=";")